# Classes

Comme on l'a vu, en python, tout est objet.

Les classes sont incontournables. Elles permettent de structure "proprement" un projet, en encapsulant des valeurs et des fonctionnalités au sein d'une même entité.

Exemple

In [5]:
class Counter:
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += 1
        return self.count
    
counter = Counter()
counter.reset(0) 
print(counter.add())
print(counter.add())
print(counter.add())

1
2
3


Commentaires :
- Mot clef `class`
- Les methodes sont des fonctions dont le premier argument est `self`
- `self` represente l'objet, instance de la classe
- `self` a des attributs, accessibles par l'opérateur `.` ils peuvent être ajoutés et modifiés **sans contraintes**
- Une classe est instanciée par son appel (`counter = Counter()`)

Les attributs peuvent être aussi modifiés et ajoutés depuis l'objet lui même !

In [6]:
obj1 = Counter()
obj1.x = 5
x = 7
print("x in object 1 =",obj1.x)

x in object 1 = 5


### Initialisation

In [8]:
class Counter:
    
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count
    

`__init__`, qui se prononce "dunder init" est apellé au moment de l'instanciation. 

In [11]:
c = Counter(step=2)
print(c.add())
print(c.add())
print(c.add())

2
4
6


**dir()** est utile pour connaître les attributs et méthodes accessibles

In [15]:
print("Contents of Counter class:",dir(Counter) )
print("------------------------")
print("------------------------")
print("Contents of counter object:", dir(c))

Contents of Counter class: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add', 'reset']
------------------------
------------------------
Contents of counter object: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add', 'count', 'reset', 'step']


## `__dict__`

`__dict__` est le dictionnaire des attributs de l'object. 

On peut le consulter ou le manipuler directement.

In [51]:
print(c.__dict__)
c.newattr = "newval"
print(c.__dict__)
print(c.__dict__["newattr"])
c.__dict__["newattr"] = "another val"
print(c.newattr)
print(c.__dict__)

{'name': 'Nishanth', 'age': 21, 'money': 60000, 'job': 'Dancer'}
{'name': 'Nishanth', 'age': 21, 'money': 60000, 'job': 'Dancer', 'newattr': 'newval'}
newval
another val
{'name': 'Nishanth', 'age': 21, 'money': 60000, 'job': 'Dancer', 'newattr': 'another val'}


On voit ici que les attributs d'un object sont implémentés comme un dictionnaire python. Cela a des coûts en taille et en performance. 

Si l'ensemble des attributs possible est fixé d'avance, il est possible d'empecher python d'utiliser un dictionnaire, par l'utilisation de `__slots__`.

In [53]:
class Counter:
    __slots__= ("step", "count")
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count

In [56]:
# Counter().__dict__   # AttributeError : 'Counter' object has no attribute '__dict__'
# Counter().newattr = "newval"  # AttributeError: 'Counter' object has no attribute 'newattr'

AttributeError: 'Counter' object has no attribute 'newattr'

Plus d'informations [ici](https://stackoverflow.com/questions/472000/usage-of-slots)

Nous reconnaissons les attributs et méthodes que nous avons défini (add, count, reset, step, `__init__`) mais aussi un certain nombres de méthodes et attributs herités du type `Object`. 

Ces méthodes sont utilisés selon le contexte d'utilisation de l'objet. Quelques exemples :

- `__hash__()` invoqué en interne pour obtenir un hash de l'objet (utile dans le cadre de sets ou de dictionnaires)
- `__getattribute__()` invoqué lorsqu'on recherche un attribut.
- `__str__()` invoqué lors de str(monobjet) ou print(monobjet)
- `__repr__()` utilisé à la place de `__str__()` si ce dernier n'est pas défini. Vise à fournir sous forme de texte une représentation plus précise que `__str__()`.

Il est possible, et parfois utile, de les redéfinir soi même (`__init__` par exemple).

Beaucoup d'autres dunders methodes (autre que celles déjà présentes) peuvent être définis pour augmenter les fonctionnalités de la classe :

- `__get_attr__` invoqué lorsqu'un attribut est introuvable
- `__iter__` invoqué lorsqu'on veut itérer sur votre object (comme dans le cadre d'une boucle for)
- `__len__` invoqué lors de len(monobjet)


Les dunders ne doivent pas être considérés comme des attributs ou méthodes privées. D'ailleurs le concept n'existe pas en python. Mais plutôt comme des méthodes utiles (on les apelle d'ailleurs parfois "magic methods").


Le sujet des dunder methods est riche et vaste. Liens pour en savoir plus :
- [Documentation](https://docs.python.org/3/reference/datamodel.html) . 
- [Excellente vidéo](https://www.youtube.com/watch?v=M4gPxbo6G6k) de l'auteur du livre du livre Fluent Python, portant notamment sur l'utilité des dunders.

## Heritage

Python suporte l'héritage

In [30]:
class SoftwareEngineer:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def salary(self, value):
        self.money = value
        print(self.name,"earns",self.money)

In [31]:
a = SoftwareEngineer('Kartik',26)

In [32]:
a.salary(40000)

Kartik earns 40000


In [33]:
[ name for name in dir(SoftwareEngineer) if not name.startswith("_")]

['salary']

Now consider another class Artist which tells us about the amount of money an artist earns and his artform.

In [34]:
class Artist:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def money(self,value):
        self.money = value
        print(self.name,"earns",self.money)
    def artform(self, job):
        self.job = job
        print(self.name,"is a", self.job)

In [35]:
b = Artist('Nitin',20)

In [36]:
b.money(50000)
b.artform('Musician')

Nitin earns 50000
Nitin is a Musician


In [37]:
[ name for name in dir(b) if not name.startswith("_")]

['age', 'artform', 'job', 'money', 'name']

money method and salary method are the same. So we can generalize the method to salary and inherit the SoftwareEngineer class to Artist class. Now the artist class becomes,

In [38]:
class Artist(SoftwareEngineer):
    def artform(self, job):
        self.job = job
        print self.name,"is a", self.job

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(self.name,"is a", self.job)? (<ipython-input-38-0810ba3114d9>, line 4)

In [39]:
c = Artist('Nishanth',21)

In [40]:
dir(Artist)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'artform',
 'money']

In [41]:
c.salary(60000)
c.artform('Dancer')

AttributeError: 'Artist' object has no attribute 'salary'

Suppose say while inheriting a particular method is not suitable for the new class. One can override this method by defining again that method with the same name inside the new class.

In [44]:
class Artist(SoftwareEngineer):
    
    def artform(self, job):
        self.job = job
        print(self.name,"is a", self.job)
        
    def salary(self, value):
        self.money = value
        print(self.name,"earns",self.money)
        print("I am overriding the SoftwareEngineer class's salary method")

In [45]:
c = Artist('Nishanth',21)

In [46]:
c.salary(60000)
c.artform('Dancer')

Nishanth earns 60000
I am overriding the SoftwareEngineer class's salary method
Nishanth is a Dancer


## Heritage multiple

L'héritage multiple est possible

In [47]:
class YellingEntity:
    def yell(self):
        print ("AAAAAHHHHHHHH")

class Artist(SoftwareEngineer, YellingEntity):
    def artform(self, job):
        self.job = job
        print(self.name,"is a", self.job)
        
    def salary(self, value):
        self.money = value
        print(self.name,"earns",self.money)
        print("I am overriding the SoftwareEngineer class's salary method")

In [50]:
anna = Artist("anna", 25)
anna.yell()

AAAAAHHHHHHHH


L'héritable multiple peut causer des résultats innatendus lors du MRO (method resolution order). Il est conseillé de n'hériter qu'une seule classe concrète (ici `SoftwareEngineer`)